Semana 5 Día 2

### AutoGen AgentChat - Profundizando...

1. Conversación multimodal
2. Salidas estructuradas
3. Uso de herramientas LangChain
4. Equipos

...y una sorpresa especial

In [ ]:
from io import BytesIO
import requests
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv(override=True)


### Una conversación multi-modal

In [ ]:
url = "https://edwarddonner.com/wp-content/uploads/2024/10/from-software-engineer-to-AI-DS.jpeg"

pil_image = Image.open(BytesIO(requests.get(url).content))
img = AGImage(pil_image)
img

In [ ]:
multi_modal_message = MultiModalMessage(content=["Describe el contenido de esta imagen en detalle", img],
                                         source="User")

In [ ]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

describer = AssistantAgent(
    name="description_agent",
    model_client=model_client,
    system_message="Eres bueno describiendo imágenes",
)

response = await describer.on_messages([multi_modal_message], cancellation_token=CancellationToken())
reply = response.chat_message.content
display(Markdown(reply))

### Salidas Estructuradas!

Autogen AgentChat hace que sea muy sencillo.

In [ ]:

class ImageDescription(BaseModel):
    scene: str = Field(description="Brevemente, el escenario general de la imagen")
    message: str = Field(description="El punto que la imagen está tratando de transmitir")
    style: str = Field(description="El estilo artístico de la imagen")
    orientation: Literal["portrait", "landscape", "square"] = Field(description="La orientación de la imagen")


In [ ]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

describer = AssistantAgent(
    name="description_agent",
    model_client=model_client,
    system_message="Eres bueno describiendo imágenes en detalle",
    output_content_type=ImageDescription,
)

response = await describer.on_messages([multi_modal_message], cancellation_token=CancellationToken())
reply = response.chat_message.content
reply

In [ ]:
import textwrap
print(f"Escenario:\n{textwrap.fill(reply.scene)}\n\n")
print(f"Mensaje:\n{textwrap.fill(reply.message)}\n\n")
print(f"Estilo:\n{textwrap.fill(reply.style)}\n\n")
print(f"Orientación:\n{textwrap.fill(reply.orientation)}\n\n")

### Usando las herramientas de LangChain desde AutoGen

In [ ]:
# AutoGen's wrapper:

from autogen_ext.tools.langchain import LangChainToolAdapter

# LangChain tools:

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain.agents import Tool


prompt = """Tu tarea es encontrar un vuelo directo de ida de JFK a LHR en junio de 2025.
Primero, busca en línea ofertas atractivas.
Después, anota todas las ofertas en un archivo llamado flights.md con todos los detalles.
Finalmente, selecciona la que consideres mejor y responde con un breve resumen.
Responde solo con el vuelo seleccionado, y solo después de haber escrito los detalles en el archivo."""


serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Útil para cuando necesitas buscar en Internet.")
autogen_serper = LangChainToolAdapter(langchain_serper)
autogen_tools = [autogen_serper]

langchain_file_management_tools = FileManagementToolkit(root_dir="sandbox").get_tools()
for tool in langchain_file_management_tools:
    autogen_tools.append(LangChainToolAdapter(tool))

for tool in autogen_tools:
    print(tool.name, tool.description)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="searcher", model_client=model_client, tools=autogen_tools, reflect_on_tool_use=True)
message = TextMessage(content=prompt, source="user")
result = await agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))

In [ ]:
# Ahora necesitamos llamar nuevamente al agente para escribir el archivo.

message = TextMessage(content="OK procedemos", source="user")

result = await agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))

### Interacciones con el Equipo

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import  TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool

serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Útil para cuando necesitas buscar en Internet.")
autogen_serper = LangChainToolAdapter(langchain_serper)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")


prompt = """Busca un vuelo directo de ida de JFK a LHR en junio de 2025."""


primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    tools=[autogen_serper],
    system_message="Eres un asistente de IA que busca ofertas prometedoras en vuelos. Incorpora cualquier retroalimentación que recibas.",
)

evaluation_agent = AssistantAgent(
    "evaluator",
    model_client=model_client,
    system_message="Proporciona retroalimentación constructiva. Responde con 'APROBAR' cuando tu retroalimentación sea abordada.",
)

text_termination = TextMentionTermination("APROBAR")

# Gracias a Peter A por agregar el max_turns - de lo contrario, esto podría entrar en un bucle..

team = RoundRobinGroupChat([primary_agent, evaluation_agent], termination_condition=text_termination, max_turns=20)



In [ ]:
result = await team.run(task=prompt)
for message in result.messages:
    print(f"{message.source}:\n{message.content}\n\n")


### Redoble..

## Introducimos el MCP!

Nuestro primer vistazo al Protocolo de Contexto de Modelo de Anthropic:

Autogen facilita el uso de herramientas MCP, al igual que las herramientas LangChain.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Un momento, un problema considerable para los usuarios de PC con Windows</h2>
<span style="color:#ff7800;">Tengo malas noticias. Hay un problema al ejecutar servidores MCP en PC con Windows; Mac y Linux funcionan correctamente. Este problema se conoce desde el 4 de mayo de 2025. Le pedí a o3 de Deep Research que buscara soluciones alternativas; <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmaron el problema</a> y la solución alternativa.<br/><br/>
La solución alternativa es un poco aburrida. Se trata de aprovechar "WSL", el enfoque de Microsoft para ejecutar Linux en tu PC. ¡Tendrás que seguir más instrucciones de configuración! Pero es rápido, y varios estudiantes han confirmado que les funciona a la perfección. Entonces, este laboratorio y los laboratorios MCP de la semana 6 funcionan. Además, WSL es una excelente manera de desarrollar software en tu PC con Windows. También puedes omitir esta última celda, pero tendrás que volver a ella al comenzar la semana 6.<br/>
Las instrucciones de configuración de WSL se encuentran en la carpeta Setup, <a href="../setup/SETUP-WSL.md">en el archivo llamado SETUP-WSL.md</a>. Espero que esto solo te retrase un poco; deberías estar listo para usar pronto. ¡Qué alegría trabajar con tecnología de vanguardia!<br/><br/>
Muchas gracias al estudiante Kaushik R. por mencionar que esto es necesario aquí, así como en la semana 6. ¡Gracias, Kaushik!
</span>
        </td>
    </tr>
</table>

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# Obtenemos la herramienta fetch desde mcp-server-fetch.
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])
fetcher = await mcp_server_tools(fetch_mcp_server)

# Creamos un agente que puede usar la herramienta fetch.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="fetcher", model_client=model_client, tools=fetcher, reflect_on_tool_use=True)  # type: ignore

# Hacemos que el agente busque el contenido de una URL y lo resuma.
result = await agent.run(task="Revisa cursos.frogamesformacion.com y resume lo que aprendes. Responde en Markdown.")
display(Markdown(result.messages[-1].content))